##  15. 콘텍스트 관리자와 else 블록

### 15.1 이것 다음에 저것: if 문 이외에서의 else 블록
else 절은 if 문뿐만 아니라 for, while, try문에서도 사용할 수 있다. 이 기능은 잘 알려져 있지 않다. 규칙은 다음과 같다.
+ for 루프가 완전히 실행된 후에 (break 문으로 중간에 멈추지 않고) else 블록이 실행된다.
+ while 조건식이 거짓이 되어 while로프를 빠져나온 후에 else 블록이 실행된다.
+ try 블록에서 예외가 발생하지 않을 때만 else블록이 실행된다. 그리고 else 블록에서 발생한 예외는 else 블록 앞에 나오는 except 블록에서 처리되지 않는다.

그러므로 예외, return, break, continue문이 복합문의 주요 블록을 빠져나오게 만들면 else블록은 실행되지 않는다.

```
# 일반적인 루프에서의 else 패턴
for item in my_list
    if item.flavor == 'banana':
        break
else:
    raise ValueError('No banana flavor found!')
```

```
try:
    dangerour_call()
    # after_call() # 이 위치는 추천하지 않음
except OSError:
    log('OSError...')
else:
    after_call() # try 블록은 after_call()이 아니라 dangerous_call()에서 발생할 수 있는 에러를 처리하기 위한 것임을 명확히 알 수 있음
```

파이선에서는 try/except를 예외 처리뿐만 아니라 일반적인 제어 흐름을 구현하기 위해서도 많이 사용한다. 파이선 용어에서는 이에 대한 슬로건을 약자로 만들어 문서화까지 해놓았다.
```
EAFP(Easier to Ask Forgiveness than Permission, 허락을 구하기보다 용서를 구하는 것이 더 쉽다.)
올바른 키나 속성이 있다고 가정하고, 그 가정이 잘못되었을 때는 예외를 잡아서 처리하는 파이선 코딩 스타일을 잘 나타낸다. 깔끔하고 빠른 이 스타일을 적용한 코드에서는 try/except 문을 많이 볼 수 있다. 이 기법은 C 등 다른 언어에서 흔히 볼 수 있는 LBYL 스타일과 대비된다.
```

```
LBYL(Leap Before You leap, 누울 자리를 보고 다리를 뻗으라)
호출이나 조회를 하기 전에 전제 조건을 명시적으로 검사한다. 이 스타일은 EAFP와 대조적이며 if문을 아주 많이 사용하는 특징이 있다. 다중 쓰레드 환경에서 LBYL 스타일은 '보는' 단계와 '뻗는' 단계 사이에서 경쟁조건이 발생할 위험이 있다. 예를 들어 if key in mapping: return mapping[key] 코드는 key를 검사한 후 매핑을 조회하기 전에 다른 쓰레드에서 key를 제거하면 문제가 발생한다. 이런 문제는 락을 사용하거나 EAFP 접근법을 이용해서 해결할 수 있다.
```

### 15.2 콘텍스트 관리자와 with 블록
콘텍스트 관리자 객체는 with 문을 제어하기 위해 존재한다. with문은 try/finally 패턴을 단순화하기 위해 설계되었다. 일반적으로 finally 절 안에 있는 코드는 중요한 리소스를 해제하거나 임시로 변경된 상태를 복원하기 위해 사용된다. 콘텍스트 관리자 프로토콜은 \_\_enter\_\_( ), \_\_exit\_\_( ) 메서드로 구성된다.

In [1]:
""" 예제 15-1 콘텍스트 관리자로서 파일 객체의 사용 예 """
with open('bingo.py') as fp:
    src = fp.read(60)

print(len(src)) 
print(fp)                     # with가 종료된 후에도 변수는 살아있음
print(fp.closed, fp.encoding) # fp의 속성을 읽을 수 있음
print(fp.read(60))            # with 블록이 끝날 때 texIOWrapper.__exit__() 메서드가 호출되어 파일을 닫으므로, fp를 이용해서 파일 입출력을 할 수 없다.

60
<_io.TextIOWrapper name='bingo.py' mode='r' encoding='UTF-8'>
True UTF-8


ValueError: I/O operation on closed file.

예제 15-1에서는 open( ) 함수가 TextIOWrapper객체를 반환하고, 이 객체의 \_\_enter\_\_( ) 메서드는 self를 반환했을 뿐이다. 그러나 \_\_enter\_\_( ) 메서드는 콘텍스트 관리자 대신 다른 객체를 반환할 수도 있다. 제어 흐름이 with문을 빠져나온 후에는 \_\_enter\_\_( ) 메서드가 반환한 객체가 아니라 콘텍스트 관리자 객체의 \_\_exit\_\_( ) 메서드가 호출된다. 

with문의 as절은 선택적이다. Open( )의 경우에는 파일에 대한 참조가 필요하지만, 사용자에게 반환할 적절한 객체가 없어서 None을 반환하는 콘텍스트 관리자도 있다. 

예제 15-2는 아주 간단하지만 콘텍스트 관리지와 \_\_enter\_\_( ) 메서드가 반환하는 객체의 차이를 잘 보여준다.

In [2]:
""" [예제 15-2, 3] LookingGlass 콘텍스트 관리자 클래스의 시험 주행 """

class LookingGlass:
    """ LookingGlass 콘텍스트 관리자 클래스 코드 """
    
    def __enter__(self): # self 인수만으로 __enter__() 메서드를 호출한다.
        import sys       
        self.original_write = sys.stdout.write # 나중에 사용하기 위해 객체 속성에 원래 sys.stdout.write를 저장한다. 
        sys.stdout.write = self.reverse_write  # 멍키 패칭을 통해 직접 만든 메서드로 변경한다. 
        return 'JABBERWOCKY'
    
    def reverse_write(self, text): # 직접 만든 메서드는 text 순서를 뒤집고 원래 sys.stdout.write를 호출한다.
        self.original_write(text[::-1])
        
    def __exit__(self, exc_type, exc_value, traceback): # 정상적으로 수행이 완료되면 파이선은 None, None, None 인수로 __exit__() 메서드를 호출한다. 
        """
        exc_typev : ZeroDivisionError 등의 예외 클래스
        exc_value : 예외 객체, 예외 메시지 등 exception() 생성자에 전달된 인수는 exc_value.args 속성을 이용해서 볼 수 있다.
        traceback : traceback 객체 (※ self가 받는 세 개의 인수는 try/finally 문의 finally 블록에서 sys.exc_info()를 호출해서 받는 정보와 동일하다.)
        """
        import sys
        sys.stdout.write = self.original_write # 원래 메서드로 변경한다.
        if exc_type is ZeroDivisionError:      # exception 인수가 None이 아니고 ZeroDivisionError면 메시지를 출력한다.
            print('Please DO NOT divide by zero!')
            return True                        # True를 반환해서 예외가 처리되었음을 알린다.
        # __exit__()가 None이나 True 이외의 값을 반환하면 With 블록에서 발생한 예외가 상위 코드로 전달된다. 

with LookingGlass() as what: # LookingGlass 객체가 콘텍스트 관리자. 파이선은 콘텍스트 관리자의 __enter__() 메서드를 호출해서 반환된 값을 what에 바인딩 한다.
    print('Alice, Kitty and Snowdrop')
    print(what)

print('\nBack to normal') # with문이 끝났으니, __enter__()가 반환해서 what에 저장한 문자열 'JABBERWOCKY'를 제대로 출력할 수 있다.
print(what)              

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ

Back to normal
JABBERWOCKY


In [3]:
""" [예제 15-4] with 블록 없이 Looking Glass 사용하기 """

manager = LookingGlass()
print(repr(manager))

monster = manager.__enter__()   # 콘텍스트 관리자의 __enter__() 메서드를 호출하고 결과를 monster에 저장한다.
print(monster == 'JABBERWOCKY') # stdout.write()의 멍키 패칭에 의해, 출력 메시지 Ture가 역순으로 출력되었다. 

print(monster)
print(manager)

manager.__exit__(None, None, None) # stdout.write()를 복원한다.
print(monster)

eurT
YKCOWREBBAJ
>89f7f711efe7x0 ta tcejbo ssalGgnikooL.__niam__<
JABBERWOCKY


콘텍스트 관리자는 상당히 독특한 기능으로서, 느리지만 확실히 파이썬 거큐니티에서 이 기능을 창의적으로 활용하는 방법을 찾아내고 있다. 사용되는 예는 다음과 같다.
+ sqlite3 모듈의 트랙잭션 관리
+ threading 코드에서 lock, condition, semaphore 보관 (17.1.10절)
+ Decimal 객체의 산술 연한 환경 설정 : deciaml.localcontext
+ 객체의 테스트를 위한 임시패치 적용 : unitest.mock.patch()

### 15.3 contextlib 유틸리티

with문 콘텍스트를 위한 유틸리티를 제공한다.

+ closing( ) : close( ) 메서드는 제공하지만 \_\_enter\_\_( ), \_\_exit\_\_( ) 프로토콜을 구현하지 않은 객체로부터 콘텍스트 관리자를 생성
+ suppress : 지정한 예외를 임시로 무시하는 콘텍스트 관리자
+ @contextmanager : 클래스를 생성하고 프로토콜을 구현하는 대신 간단한 제너레이터 함수로부터 콘텍스트 관리자를 생성할 수 있게 해주는 데커레이터
+ ContextDecorator : 콘텍스트 관리자를 함수 데커레이터로도 사용할 수 있게 해주는 기반 클래스
+ ExitStack : 여러 콘텍스트 관리자를 입력할 수 있게 해주는 콘텍스트 관리자. with 블록이 끝나면 ExitStack은 누적된 콘텍스트 관리자들의 \_\_exit\_\_( ) 메서드를 LIFO(마지막에 들어간 것이 처음 나옴)로 호출한다. 예를 들어 임의의 파일 리스트에 있는 파일을 한꺼번에 여는 경우처럼 with 블록 안에 들어가기 전에 얼마나 많은 콘텍스트 관리자가 필요한지 사전에 알 수 없을 때 이 클래스를 사용해야 한다.

이 유틸리티들 중 @contextmanager 데커레이터가 가장 널리 사용되므로 자세히 살펴보자.

### 15.4 @contextmanager 사용하기

@contextmanager 데커레이터는 \_\_enter\_\_( ), \_\_exit\_\_( ) 메서드를 가진 클래스 전체를 작성하는 대신 \_\_enter\_\_( ) 메서드가 반환할 것을 생성하는 yield문 하나를 가진 제너레이터만 구현하면 된다. @contextmanager로 데커레이트된 제너레이터에서 yield는 함수 본체를 두 부분으로 나누기 위해 사용된다. yield 문 앞에 있는 모든 코드는 with 블록 앞에서 인터프리터가 \_\_enter\_\_( ) 를 호출할 때 실행되고, yield 문 뒤에 있는 코드는 블록의 마지막에서 \_\_exit\_\_( )가 호출될 때 실행된다.



In [4]:
""" [예제 15-5, 6] 제너레이터로 구현한 콘텍스트 관리자 """

import contextlib

@contextlib.contextmanager 
def looking_glass():
    import sys       
    original_write = sys.stdout.write # 나중에 사용하기 위해 객체 속성에 원래 sys.stdout.write를 저장한다. 
    
    def reverse_write(text):
        original_write(text[::-1])
    
    sys.stdout.write = reverse_write  # 멍키 패칭을 통해 직접 만든 메서드로 변경한다. 
    yield 'JABBERWOCKY'               # with 문의 as절에 있는 타킷 변수에 바인딩될 값을 생성한다. with문의 본체가 실행되는 동안 이 함수는 여기에서 실행을 일시 중단한다.

    sys.stdout.write = original_write

# 콘텍스트 관리자 함수 사용    
with looking_glass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

print('\nBack to normal')
print(what)   

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ

Back to normal
JABBERWOCKY


예제 15-5는 with블록 안에서 예외가 발생하면 looking_glass() 함수는 원래의 sys.stdout.write() 메서드를 복원하지 않고 중단하므로 시스템이 불안정한 상태로 남게 된다. 예제 15-7은 ZeroDivisoinError 에외를 특별히 처리해서 예제 15-3과 동일하게 작동한다.

예제 15-8은 콘텍스트 관리자를 사용하는 방법을 보여준다.

In [5]:
""" [예제 15-5, 6] 제너레이터로 구현한 콘텍스트 관리자 """

import contextlib

@contextlib.contextmanager 
def looking_glass():
    import sys       
    original_write = sys.stdout.write # 나중에 사용하기 위해 객체 속성에 원래 sys.stdout.write를 저장한다. 
    
    def reverse_write(text):
        original_write(text[::-1])
    
    sys.stdout.write = reverse_write  # 멍키 패칭을 통해 직접 만든 메서드로 변경한다. 
    
    msg = ''
    
    """ Tip : contextmanager를 사용할 때는 어쩔 수 없이 yield 문 주변을 try/finally나 with 블록으로 둘러싸야 한다. 
              콘텍스트 관리자의 사용자가 자신의 with 블록 안에서 어떤 일을 할지 모르기 때문이다. """
    try: 
        yield 'JABBERWOCKY'
    except ZeroDivisionError:
        msg = 'Please DO NOT divide by zero!'
    finally:
        sys.stdout.write = original_write
        if msg:
            print(msg)

# 콘텍스트 관리자 함수 사용    
with looking_glass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

print('\nBack to normal')
print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ

Back to normal
JABBERWOCKY


In [ ]:
""" [예제 15-8] 파일을 덮어쓰기 위한 콘텍스트 관리자
               http://www.zopatista.com/python/2013/11/26/inplace-file-rewriting/
"""

from contextlib import contextmanager
import io
import os


@contextmanager
def inplace(filename, mode='r', buffering=-1, encoding=None, errors=None,
            newline=None, backup_extension=None):
    """
    Allow for a file to be replaced with new content.

    yields a tuple of (readable, writable) file objects, where writable
    replaces readable.

    If an exception occurs, the old file is restored, removing the
    written data.

    mode should *not* use 'w', 'a' or '+'; only read-only-modes are supported.
    """

    # move existing file to backup, create new file with same permissions
    # borrowed extensively from the fileinput module
    if set(mode).intersection('wa+'): # 교집합(w, a, +가 있다면)
        raise ValueError('Only read-only file modes can be used')

    backupfilename = filename + (backup_extension or os.extsep + 'bak')
    try:
        os.unlink(backupfilename)
    except os.error:
        pass
    os.rename(filename, backupfilename)
    readable = io.open(backupfilename, mode, buffering=buffering,
                       encoding=encoding, errors=errors, newline=newline)
    try:
        perm = os.fstat(readable.fileno()).st_mode
    except OSError:
        writable = open(filename, 'w' + mode.replace('r', ''),
                        buffering=buffering, encoding=encoding, errors=errors,
                        newline=newline)
    else:
        os_mode = os.O_CREAT | os.O_WRONLY | os.O_TRUNC
        if hasattr(os, 'O_BINARY'):
            os_mode |= os.O_BINARY
        fd = os.open(filename, os_mode, perm)
        writable = io.open(fd, "w" + mode.replace('r', ''), buffering=buffering,
                           encoding=encoding, errors=errors, newline=newline)
        try:
            if hasattr(os, 'chmod'):
                os.chmod(filename, perm)
        except OSError:
            pass
    try:
        yield readable, writable
    except Exception:
        # move backup back
        try:
            os.unlink(filename)
        except os.error:
            pass
        os.rename(backupfilename, filename)
        raise
    finally:
        readable.close()
        writable.close()
        try:
            os.unlink(backupfilename)
        except os.error:
            pass

# 파일을 덥어쓰기 위한 콘텍스트 관리자 실행
import csv

with inplace(csvfilename, 'r', newline='') as (infh, outfh):
    reader = csv.reader(infh)
    writer = csv.writer(outfh)
    
    for now in reader:
        row += ['new', 'columns']
        writer.writerow(row)